## 0. Environment

> Uncomment these lines only if necessary!

- Venv
- Data decompression

### 0.1 Install python libraries 
> Install pandas and openpyxl in a conda venv

SHELL
```shell
conda create --name pns_venv --yes
conda activate /workspaces/depressao-ibge-pns/pns_venv
conda install pandas --yes
conda install openpyxl --yes
```

In [1]:
# # Uncomment the following lines to unzip the compressed pns-data into the specified directory

# %%bash
# conda create --name pns_venv --yes
# conda activate /workspaces/depressao-ibge-pns/pns_venv
# conda install pandas --yes
# conda install openpyxl --yes

### 0.2 Decompress the pns-database and the dictionary
> The data is include in data/pns-original/compressed-data.zip ~40MB<br>
> Decompresses it and you will have up to ~500MB

SHELL
```shell
export COMPRESSED_PATH="/workspaces/depressao-ibge-pns/data/pns-original/compressed-data.zip"
export DESTINATION_PATH="/workspaces/depressao-ibge-pns/data/pns-original/"
unzip $(COMPRESSED_PATH) -d $(DESTINATION_PATH) -x "__MACOSX/*"
```

In [2]:
# # Uncomment the following lines to unzip the compressed pns-data into the specified directory

# %%bash
# export COMPRESSED_PATH="/workspaces/depressao-ibge-pns/data/pns-original/compressed-data.zip"
# export DESTINATION_PATH="/workspaces/depressao-ibge-pns/data/pns-original/"
# unzip "$COMPRESSED_PATH" -d "$DESTINATION_PATH" -x "__MACOSX/*"

## 1. Setup
- Lib Imports
- File paths
- `pns-dictionary.csv`
- Column widths

### 1.1 Imports

In [1]:
# Libraries
import pandas as pd
import numpy as np

### 1.2 File paths

In [2]:
pns_data_path = "../data/pns-original/pns-2019.txt"
dictionary_path = "../data/pns-original/pns-dictionary-clean.xlsx"

### 1.3 Dictionay

#### 1.3.1 Read the excel dictionary

In [3]:
dictionary = pd.read_excel(dictionary_path, skiprows=2)
dictionary.iloc[2:30]

,ModuloParteAnotacao,PosiçãoInicial,Tamanho,Codigo,Numero,Descricao,Tipo,Categoria
2,NaN,1.0,2.0,V0001,NaN,Unidade da Federação,11,Rondônia
3,NaN,NaN,NaN,NaN,NaN,NaN,12,Acre
4,NaN,NaN,NaN,NaN,NaN,NaN,13,Amazonas
5,NaN,NaN,NaN,NaN,NaN,NaN,14,Roraima
6,NaN,NaN,NaN,NaN,NaN,NaN,15,Pará
7,NaN,NaN,NaN,NaN,NaN,NaN,16,Amapá
8,NaN,NaN,NaN,NaN,NaN,NaN,17,Tocantins
9,NaN,NaN,NaN,NaN,NaN,NaN,21,Maranhão
10,NaN,NaN,NaN,NaN,NaN,NaN,22,Piauí
11,NaN,NaN,NaN,NaN,NaN,NaN,23,Ceará


In [4]:
# Save the 1088 descriptions of the variables in the dictionary
descriptions = dictionary.Descricao.dropna(axis=0).str.strip()

#### 1.3.2 Improve the dictionary

In [5]:
# Remove escape sequences
dictionary['Descricao'] = dictionary['Descricao'].str.replace(r"\\n", "", regex=True).str.strip()

In [6]:
# Excel merged cells become NaN in a dataframe
# Fill NaN values in the 'Descricao', 'ModuloParteAnotacao', 'Codigo' and 'Numero' columns
for column in ['Descricao', 'ModuloParteAnotacao', 'Codigo', 'Numero']:
    dictionary[column] = dictionary[column].fillna(method='ffill')

/tmp/ipykernel_2528/1141008394.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dictionary[column] = dictionary[column].fillna(method='ffill')


#### 1.3.3 Save it as csv

In [7]:
# Uncomment to save the improved dictionary as a CSV file
dictionary.to_csv("../data/pns-dictionary.csv", index=False)

# Display the first few rows to confirm changes
display(dictionary.shape, dictionary.iloc[35-14:35-4])

(5221, 8)

,ModuloParteAnotacao,PosiçãoInicial,Tamanho,Codigo,Numero,Descricao,Tipo,Categoria
21,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,35,São Paulo
22,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,41,Paraná
23,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,42,Santa Catarina
24,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,43,Rio Grande do Sul
25,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,50,Mato Grosso do Sul
26,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,51,Mato Grosso
27,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,52,Goiás
28,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,53,Distrito Federal
29,Parte 1 - Identificação e Controle,3.0,7.0,V0024,NaN,Estrato,NaN,NaN
30,Parte 1 - Identificação e Controle,10.0,9.0,UPA_PNS,NaN,UPA,NaN,NaN


### 1.4 Widths

In [11]:
column_widths = dictionary.Tamanho.dropna(axis=0).values.astype(int)
print(
    column_widths[:10],
    column_widths[1088-10:1088],
)

[2 7 9 4 2 4 2 1 1 1] [1 2 8 8 1 2 1 1 1 8]


## 2. PNS Data Frame
- FWF
- Column descriptions

### 2.1 PNS DF using FWF 

In [12]:
# FWF
pns = pd.read_fwf(
    pns_data_path,
    header=None,
    dtype=str,
    widths=column_widths
)

In [13]:
display(
    pns.shape,
    pns.head()
)

(293726, 1088)

,0,1,2,3,4,5,6,7,8,9,...,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087
0,11,1110011,110000016,0001,01,2019,06,1,1,1,...,1,06,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
1,11,1110011,110000016,0001,01,2019,06,1,1,0,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
2,11,1110011,110000016,0001,01,2019,06,1,1,0,...,1,04,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
3,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
4,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504


### 2.2 Name the PNS DF columns with descriptions from the dictionary

In [14]:
pns.columns = descriptions
pns.head()

Descricao,Unidade da Federação,Estrato,UPA,Número de ordem do domicílio na PNS,Tipo da entrevista,Ano de referência,Total de moradores,Tipo de situação censitária,Tipo de área,Seleção do morador de 15 anos ou mais para responder o questionário individual,...,Condição de ocupação na semana de referência para pessoas de 14 anos ou mais de idade,Grupamentos de atividade do trabalho principal da semana de referência para pessoas de 14 anos ou mais de idade,"Rendimento domiciliar \n(exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)","Rendimento domiciliar per capita\n(exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)","Faixa de rendimento domiciliar per capita (exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)",Idade em meses das crianças nascidas no período de referência do Módulo L,Faixa de tempo gasto por dia no deslocamento casa-trabalho pelas pessoas ocupadas que se deslocavam para o trabalho,Tipo de cigarro industrializado comprado,Método contraceptivo mais eficaz que faz uso,"Data de geração do arquivo de microdados. Data ordenada na forma: ano (4 algarismos), mês (2) e dia (2) - AAAAMMDD"
0,11,1110011,110000016,0001,01,2019,06,1,1,1,...,1,06,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
1,11,1110011,110000016,0001,01,2019,06,1,1,0,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
2,11,1110011,110000016,0001,01,2019,06,1,1,0,...,1,04,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
3,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
4,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504


## 3. Attribute selection
- Read attribute-selection.txt file
- Remove escape sequences
- `depression.csv`

### 3.1 Read attribute selection

In [15]:
# read data/attribute-selection.txt into a python str list
with open("../data/attribute-selection.txt", "r") as file:
    selected_attributes = [line.replace("\\n", "").strip() for line in file.readlines()]

# show me the duplicates
duplicates = [item for item, count in zip(selected_attributes, np.bincount([selected_attributes.index(i) for i in selected_attributes])) if count > 1]
display(f"Duplicates: {len(duplicates)}", pd.DataFrame(duplicates))

# Remove duplicates from selected_attributes
selected_attributes = list(dict.fromkeys(selected_attributes))
display(f"Selected Attributes: {len(selected_attributes)}", pd.DataFrame(selected_attributes).head())

'Duplicates: 0'

""


'Selected Attributes: 53'

,0
0,"Nas duas últimas semanas, com que frequência o..."
1,Algum médico ou profissional de saúde mental (...
2,Algum médico já lhe receitou algum medicamento...
3,Nas duas últimas semanas o(a) senhor(a) usou a...
4,Que idade o(a) Sr(a) tinha quando começou a fu...


### 3.2 remove \n escape instances from the pns.columns

In [16]:
# remove escaped sequences from the pns.columns
pns.columns = pns.columns.str.replace(r"\n", "", regex=True).str.strip()

### 3.3 Depression V1

In [17]:
depression = pns[selected_attributes]
display(
    depression.shape,
    depression.head()
)

(293726, 56)

Descricao,"Nas duas últimas semanas, com que frequência o(a) Sr(a) se sentiu deprimido(a), “pra baixo” ou sem perspectiva?",Algum médico ou profissional de saúde mental (como psiquiatra ou psicólogo) já lhe deu o diagnóstico de depressão?,Algum médico já lhe receitou algum medicamento para depressão?,Nas duas últimas semanas o(a) senhor(a) usou algum medicamento para depressão?,Que idade o(a) Sr(a) tinha quando começou a fumar produto de tabaco diariamente?,O(A) Sr(a) vai ao médico/serviço de saúde regularmente por causa da depressão ou só quando tem algum problema?,Qual o principal motivo do(a) Sr(a) não visitar o médico/serviço de saúde regularmente por causa da depressão?,Por causa da depressão Faz psicoterapia,Por causa da depressão Toma medicamentos,Algum dos medicamentos para depressão foi obtido em serviço público de saúde?,...,Idade do morador na data de referência,Unidade da Federação,Tipo de área,Tipo de situação censitária,Cor ou raça,Qual é a sua orientação sexual?,Qual foi o curso mais elevado que ___frequentou,"Quem fez isso com você? (Se mais de uma pessoa, defina o pirncipal agressor)",Quem fez isso com você,Quem fez isso com você
0,2,1,1,3,15,2,01,2,1,1,...,055,11,1,1,1,1,05,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,069,11,1,1,4,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,031,11,1,1,2,NaN,10,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,009,11,1,1,2,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,006,11,1,1,4,NaN,NaN,NaN,NaN,NaN


## 4. Save to .csv

In [ ]:
# # Uncomment the following two lines to save
# output_path = "../data/depression.csv"
# depression.to_csv(output_path, index=False)